Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# План работы:

1) [Изучить общую информацию.](#id_1)

2) [Исследовать задачу.](#id_2)

3) [Работа с дисбалансом.](#id_3)

4) [Тестирование модели.](#id_4)

5) [Общий вывод.](#id_5)

<a id='id_1'></a>
# 1. Подготовка данных

In [3]:
# импортируем библиотеки, ознакамливаемся с данными
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
data = pd.read_csv('/datasets/Churn.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Итого - у нас есть 14 колонок и 10000 строк. 

___Целевой признак___ - столбец Exited, все остальное - ___признаки___. 

Столбец __Surname__ - фамилия клиента, модель он может только запутать, для расчетов он не пригодится., поэтому создадим отдельный словарь, а из общего датафрейма столбец уберем. Аналогично поступим и со столбцами __RowNumber__ и __CustomerId__

Так же есть пропуски в столбце __Tenure__ (количество недвижимости у объекта) - скорее всего, данные просто не были внесены, предположительно из-за нулевого значения. (при возможности - запрашиваем недостающую информацию, сейчас же заменим пропуски на 0). Так же переведем столбец в формат _int_.

In [4]:
# Уберем все пропуски, заменим тип столбца Tenure на int
data['Tenure'] = data['Tenure'].fillna(0).astype(int)
data.info(5)
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# создадим словарь
data_dict = data[{'RowNumber', 'CustomerId', 'Surname'}]
data_dict.head()

,CustomerId,Surname,RowNumber
0,15634602,Hargrave,1
1,15647311,Hill,2
2,15619304,Onio,3
3,15701354,Boni,4
4,15737888,Mitchell,5


In [6]:
# создадим отдельный датафрейм с интересующими нас столбцами
data_model = data.drop({'RowNumber', 'CustomerId', 'Surname'}, axis=1)
data_model.head(15)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


Теперь таблица готова к анализу и обработке. А значит - приступаем.

<a id='id_2'></a>
# 2. Исследование задачи

### 2.1 Исследование балансов признаков.

In [7]:
data_model.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.543400,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,3.111573,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,4.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [8]:
data_model.head(15)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


Как было известно заранее - целевой признак = Exited, все остальное - признаки, которые будут использованы для обучения.

Можно заметить сильную несбалансированность признаков. Например:

    - Признаки IsActiveMember, HasCrCard, NumOfProducts и Tenure имееют малый диапозон (0-1, 0-1, 1-4, 0-10 соотвественно).
    
    - Признак Age имеет средний диапозон (18-92)
    
    - Признаки CreditScore, Balance, EstimatedSalary имеют большой диапозон (350-850, 0-250898, 11.58 - 199992.48)
    
Так же признаки у нас идут разные: категориальные и количественные.
Категориальные - Geogreaphy и Gender, все остальные - количественные. 

А значит - для успешной работы модели категориальные признаки нам нужно будет преобразовать в количественные.

### 2.2 Преобразование признаков
Для преобразования данных воспользуемся техникой One-Hot Encoding.
Разумно было бы так же использовать Ordinal Encoder, но оно не подходит для Логистической регрессии.

In [9]:
# Чтобы избежать дамми-ловушек удалим первый столбец
data_model2 = pd.get_dummies(data_model, drop_first=True)
data_model2.head(15)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0
5,645,44,8,113755.78,2,1,0,149756.71,1,0,1,1
6,822,50,7,0.00,2,1,1,10062.80,0,0,0,1
7,376,29,4,115046.74,4,1,0,119346.88,1,1,0,0
8,501,44,4,142051.07,2,0,1,74940.50,0,0,0,1
9,684,27,2,134603.88,1,1,1,71725.73,0,0,0,1


Готово, теперь все категориальные признаки стали количественными.

### 2.3 Создание выборок

In [10]:
# Создадим обучающую, валидационную и тестовую выборки с соотношением (3:1:1).
from sklearn.model_selection import train_test_split
data_train, data_another = train_test_split(data_model2, test_size = 0.4, random_state=123)
data_valid, data_test = train_test_split(data_another, test_size=0.5, random_state=123)
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6000 entries, 9696 to 3582
Data columns (total 12 columns):
CreditScore          6000 non-null int64
Age                  6000 non-null int64
Tenure               6000 non-null int64
Balance              6000 non-null float64
NumOfProducts        6000 non-null int64
HasCrCard            6000 non-null int64
IsActiveMember       6000 non-null int64
EstimatedSalary      6000 non-null float64
Exited               6000 non-null int64
Geography_Germany    6000 non-null uint8
Geography_Spain      6000 non-null uint8
Gender_Male          6000 non-null uint8
dtypes: float64(2), int64(7), uint8(3)
memory usage: 486.3 KB


In [11]:
data_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 2019 to 7860
Data columns (total 12 columns):
CreditScore          2000 non-null int64
Age                  2000 non-null int64
Tenure               2000 non-null int64
Balance              2000 non-null float64
NumOfProducts        2000 non-null int64
HasCrCard            2000 non-null int64
IsActiveMember       2000 non-null int64
EstimatedSalary      2000 non-null float64
Exited               2000 non-null int64
Geography_Germany    2000 non-null uint8
Geography_Spain      2000 non-null uint8
Gender_Male          2000 non-null uint8
dtypes: float64(2), int64(7), uint8(3)
memory usage: 162.1 KB


In [12]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1314 to 7718
Data columns (total 12 columns):
CreditScore          2000 non-null int64
Age                  2000 non-null int64
Tenure               2000 non-null int64
Balance              2000 non-null float64
NumOfProducts        2000 non-null int64
HasCrCard            2000 non-null int64
IsActiveMember       2000 non-null int64
EstimatedSalary      2000 non-null float64
Exited               2000 non-null int64
Geography_Germany    2000 non-null uint8
Geography_Spain      2000 non-null uint8
Gender_Male          2000 non-null uint8
dtypes: float64(2), int64(7), uint8(3)
memory usage: 162.1 KB


In [13]:
# Для последующего обучения моделей - создадим переменные для признаков и целевого признака для каждой из выборок
features_train = data_train.drop(['Exited'], axis=1)
target_train = data_train['Exited']
features_valid = data_valid.drop(['Exited'], axis=1)
target_valid = data_valid['Exited']
features_test = data_test.drop(['Exited'], axis=1)
target_test = data_test['Exited']

Итого - у нас получились три выборки, которые мы разделили на признаки и целевой признак:

    1) features_train, target_train составляют 60% от общих данных и являются обучающими данными
    
    2) features_valid, target_valid составляют 20% от общих данных и являются валидационными данными
    
    3) features_test, target_test составляют 20% от общих данных и являются тестовыми данными.

### 2.4 Обучение модели

Перед нами задача классификации, т.к. целевой признак равен 0 либо 1.

Попробуем 3 основных типа моделей: 

    1) Дерево решений
    
    2) Случайный лес
    
    3) Логистическая регрессия

### Дерево решений

In [14]:
# Импортируем необходимые библиотеки
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [15]:
# Для начала обучим модель со стандартными настройками
tree_model = DecisionTreeClassifier(random_state=123)
tree_model.fit(features_train, target_train)
tree_predict = tree_model.predict(features_valid)
accuracy_score(target_valid, tree_predict)

0.798

Accuracy = 0.798
Поиграем с параметрами, чтобы найти максимально подходящую нам модель

In [16]:
# Начнем с max_depth, попробуем его в цикле от 1 до 10)
for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth=depth, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy=accuracy_score(target_valid, predict)
    print(depth, "-", accuracy)

1 - 0.803
2 - 0.846
3 - 0.8515
4 - 0.854
5 - 0.8595
6 - 0.8595
7 - 0.8625
8 - 0.8555
9 - 0.8455
10 - 0.843


In [17]:
# Итак, max_depth лучше всего делать 7, в этом случае значение самое большое
# Теперь попробуем выбрать min_samples_split 
for split in range(2, 11):
    model=DecisionTreeClassifier(max_depth=7, min_samples_split=split, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    print(split, '-', accuracy)

2 - 0.8625
3 - 0.8625
4 - 0.8625
5 - 0.8625
6 - 0.8625
7 - 0.8625
8 - 0.8625
9 - 0.8625
10 - 0.8625


In [18]:
# Какой бы мы параметр не выбрали - значение не меняется, а значит min_samples_split оставляем без изменений
# Теперь обработаем min_samples_leaf
for leaf in range (1, 11):
    model=DecisionTreeClassifier(max_depth=7, min_samples_leaf=leaf, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    print(leaf, '-', accuracy)

1 - 0.8625
2 - 0.863
3 - 0.863
4 - 0.8635
5 - 0.8635
6 - 0.863
7 - 0.8625
8 - 0.8635
9 - 0.861
10 - 0.8605


In [19]:
# Создаем финальную модель
tree_model = DecisionTreeClassifier(max_depth=7, min_samples_leaf=4, random_state=123)
tree_model.fit(features_train, target_train)
tree_predict = tree_model.predict(features_valid)
accuracy_score(target_valid, tree_predict)

0.8635

Дерево решений с оптимальным параметрами готовo, accuracy составило 0.8635 

### Случайный лес

In [20]:
from sklearn.ensemble import RandomForestClassifier
# По аналогии с моделью Дерево решений подбираем параметры
for depth in range(1, 11):
    model = RandomForestClassifier(max_depth=depth, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy=accuracy_score(target_valid, predict)
    print(depth, "-", accuracy)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1 - 0.803
2 - 0.821
3 - 0.8285


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


4 - 0.8345
5 - 0.8515
6 - 0.8485
7 - 0.8535


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


8 - 0.861
9 - 0.8605


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10 - 0.859


In [21]:
# Максимум - при значении 8, оставялем
# Теперь попробуем найти min_samples_split
for split in range(2, 11):
    model=RandomForestClassifier(max_depth=8, min_samples_split=split, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    print(split, '-', accuracy)

2 - 0.861


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


3 - 0.8595
4 - 0.859


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


5 - 0.8615
6 - 0.86


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


7 - 0.8605
8 - 0.8595


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


9 - 0.851
10 - 0.862


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [22]:
# Оставляем значение 10
# min_samples_leaf
for leaf in range (1, 11):
    model=RandomForestClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=leaf, random_state=123)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    print(leaf, '-', accuracy)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1 - 0.862


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


2 - 0.8645


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


8 - 0.8545


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


9 - 0.8565
10 - 0.8585


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [23]:
# Больше всего нам подходит 2.
# Рассмотрим n_estimators
for est in range (1, 11):
    model=RandomForestClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=2, n_estimators=est, random_state=123)
    model.fit(features_train, target_train)
    predict=model.predict(features_valid)
    accuracy=accuracy_score(target_valid, predict)
    print(est, '-', accuracy)

1 - 0.8175
2 - 0.833
3 - 0.85
4 - 0.8555
5 - 0.854
6 - 0.8585
7 - 0.862
8 - 0.859
9 - 0.859
10 - 0.8645


In [24]:
# Подходящее значение - 10
# Итак, все параметры подобраны, значит пора обучать модель
forest_model = RandomForestClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=2, n_estimators=10, random_state=123)
forest_model.fit(features_train, target_train)
forest_predict = forest_model.predict(features_valid)
accuracy_score(target_valid, forest_predict)

0.8645

Максимально подходящий случайный лес мы подобрали, результат accuracy составил 0.8645.

### Логистическая регрессия

In [25]:
# Логистическая регрессия не склонна к обучению так же, как и предыдущие модели, поэтому просто обучаем модель
from sklearn.linear_model import LogisticRegression
logistic_model=LogisticRegression(random_state=123)
logistic_model.fit(features_train, target_train)
logistic_predict=logistic_model.predict(features_valid)
accuracy_score(target_valid, logistic_predict)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7915

Accuracy = 0.7915

## Общий вывод:
Accuracy у данных моделей составило:

    - Дерево решений 0.8635
    
    - Случайный лес 0.8645
    
    - Логистическая регрессия 0.7915
    
Примерно одинаково нам подоходят и модель "Случайный лес" и модель "Дерево решений" (Дерево решений отстало всего на 0.1% правильных ответов), в то время как модель "Логистическая регрессия" проигрывает в качестве (правильных ответов на 8% меньше).

<a id='id_3'></a>
# 3. Борьба с дисбалансом

Для начала проверим предыдущие модели при помощи показателя f1 и auc_roc

In [26]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
# Дерево решений
f1_score(target_valid, tree_predict)

0.5673534072900158

In [28]:
probabilities_valid = tree_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8478808845003825

In [29]:
# Случайный лес
f1_score(target_valid, forest_predict)

0.5520661157024793

In [30]:
probabilities_valid = forest_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8523335714421174

In [31]:
# Логистическая регрессия 
f1_score(target_valid, logistic_predict)

0.06711409395973154

In [32]:
probabilities_valid = logistic_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.6347453394946616

In [33]:
# Теперь произведем масташбирование, чтобы привести все показатели к одному масштабу
from sklearn.preprocessing import StandardScaler
numeric = ['CreditScore', 'Tenure', 'Balance', 'Age', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])


<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что обучал scaler только на тренировочной выборке.
</div>

In [34]:
# Обучим Дерево решений с добавлением баланса классов
model = DecisionTreeClassifier(max_depth=7, min_samples_leaf=4, random_state=123, class_weight='balanced')
model.fit(features_train, target_train)
tree_predict = model.predict(features_valid)
f1_score(target_valid, tree_predict)

0.5730442978322337

In [35]:
accuracy_score(target_valid, tree_predict)

0.7735

In [38]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.83999168726413

In [39]:
# Аналогично для Случайного леса
model = RandomForestClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=2, n_estimators=10, random_state=123,
                              class_weight='balanced')
model.fit(features_train, target_train)
forest_predict = model.predict(features_valid)
f1_score(target_valid, forest_predict)

0.5984598459845984

In [40]:
accuracy_score(target_valid, forest_predict)

0.8175

In [42]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8533623910336238

In [43]:
# Логистическая регрессия 
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
logistic_predict = model.predict(features_valid)
f1_score(target_valid, logistic_predict)

0.45669291338582685

In [44]:
accuracy_score(target_valid, logistic_predict)

0.6895

In [46]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.7494484515553983

### Итоговый вывод:
В данном пункте мы обучали модели с учетом дисбаланса классов. Финальный результат моделей:

    1) Дерево решений - показатель f1 изменился с 0.56 до 0.57 (т.е. вырос, но всего на 0.01). roc_auc изменился с 0.85 до 0.49 (показатель сильно упал, он близок к показателю случайной модели)
    
    2) Случайный лес - показатель f1 изменился с 0.55 до 0.59 (вырос на 0.04). roc_auc изменился с 0.85 до 0.60.
    
    3) Логистическая регрессия - показатель f1 изменился с 0.06 до 0.45 (вырос на 0.39). roc_auc изменился с 0.63 до 0.72.
    
Если до изменения баланса классов самый высокий показатель f1 был у Дерева решений, то теперь лидирует Случайный лес. Но разница между ними как была небольшой, так такой и осталась (до изменений f1 у Дерева решений было выше на 0.01, после изменений выше у Случайного леса на 0.02). 

Нельзя не отметить зависимость Логистической Регрессии от баланса классов. До балансировки показатель f1 был очень маленьким (0.06), что говорит о том, что либо полнота, либо точность стремились к 0. Сейчас же показатель резко вырос на 0.39 (у остальных моделей рост был на 0.01 и 0.04). 

Относительно roc_auc - можно заметить сильный спад у Дерева решений и Случайного леса, в то время как у Логистической регрессии данный показатель вырос.

In [40]:
features_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9696,-1.036502,-0.653217,1.116306,0.329383,-0.911674,0.648425,0.964303,1.717588,0,0,1
509,-0.529560,-0.185576,-0.808267,-1.222776,0.799319,0.648425,0.964303,0.758922,0,1,0
621,0.060149,0.095009,1.757830,1.449393,-0.911674,-1.542199,0.964303,0.899918,0,1,0
7681,-0.095038,-0.559689,-0.808267,1.104228,0.799319,0.648425,0.964303,-0.762886,0,0,1
4265,0.608474,-0.653217,-0.166743,1.121965,-0.911674,-1.542199,0.964303,-1.021335,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
9785,-2.019350,-1.120858,0.154019,1.258395,0.799319,-1.542199,-1.037018,-0.503695,0,0,0
7763,-0.374374,-0.279104,-1.129029,-0.521543,-0.911674,0.648425,0.964303,-0.459497,0,1,0
5218,0.360176,-1.775556,-0.166743,0.444080,0.799319,0.648425,-1.037018,-1.058889,0,0,1
1346,-0.074346,-0.840273,0.154019,0.280512,-0.911674,0.648425,0.964303,1.137462,0,0,1


In [72]:
# Используем технику umsampling
# Для этого разделим обучающую выборку на положительные и отрицательные ответы, после чего увеличим количество 
# положительных ответов в 10 раз
features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train==0]
target_ones = target_train[target_train==1]
features_upsampled = pd.concat([features_zeros] + [features_ones]*10)
target_upsampled = pd.concat([target_zeros] + [target_ones] * 10)
# Теперь перемешаем
features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=123)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7558,-0.291607,-0.279104,-1.449791,0.511571,0.799319,0.648425,-1.037018,0.195502,0,1,0
2160,0.794698,-1.027330,0.154019,-1.222776,0.799319,-1.542199,0.964303,-1.388338,0,0,1
2574,0.153260,1.123820,1.437068,0.537841,0.799319,-1.542199,0.964303,-1.709171,1,0,0
8940,0.070494,-0.092047,0.795544,-1.222776,0.799319,0.648425,-1.037018,1.487245,0,0,1
6295,-1.191689,1.030292,-1.129029,1.010300,2.510312,-1.542199,-1.037018,-0.525218,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2384,0.846427,0.095009,1.757830,-1.222776,0.799319,0.648425,-1.037018,0.939751,0,0,0
5922,-0.984774,0.469122,-0.487505,0.460806,2.510312,0.648425,-1.037018,-0.680912,0,0,0
3610,-0.726129,1.591461,-1.129029,0.868250,-0.911674,0.648425,-1.037018,0.121757,0,0,0
3127,1.301640,-0.559689,1.116306,0.609786,-0.911674,-1.542199,-1.037018,0.512958,1,0,1


In [42]:
# Попробуем посмотреть метрики модели Дерево решений
tree_model2 = DecisionTreeClassifier(max_depth=7, min_samples_leaf=4, random_state=123, class_weight='balanced')
tree_model2.fit(features_upsampled, target_upsampled)
tree_predict2 = tree_model2.predict(features_valid)
f1_score(target_valid, tree_predict2)

0.5730550284629982

In [43]:
accuracy_score(target_valid, tree_predict2)

0.775

In [44]:
probabilities_valid = tree_model2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8380178075870308

In [45]:
# Аналогично для Случайного леса
forest_model2 = RandomForestClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=2, n_estimators=10, random_state=123,
                              class_weight='balanced')
forest_model2.fit(features_upsampled, target_upsampled)
forest_predict2 = forest_model2.predict(features_valid)
f1_score(target_valid, forest_predict2)

0.5921325051759835

In [46]:
accuracy_score(target_valid, forest_predict2)

0.803

In [47]:
probabilities_valid = forest_model2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.8539724130955617

In [48]:
# Аналогично для Логистической регрессии
logistic_model2 = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
logistic_model2.fit(features_upsampled, target_upsampled)
logistic_predict2 = logistic_model2.predict(features_valid)
f1_score(target_valid, logistic_predict2)

0.45669291338582685

In [49]:
accuracy_score(target_valid, logistic_predict2)

0.6895

In [50]:
probabilities_valid = logistic_model2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_valid, probabilities_one_valid)

0.7494373889791455

### Вывод:

Метрики у моделей после использования техники upsampled составили:

1) Дерево решений:

    - accuracy изначально показатель был 0.86, после балансировки стал 0.6, финально же составил 0.77
    
    - f1 без изменений (0.57)
    
    - roc_auc 0.84 (был 0.49)
    
Интересный факт, когда мы добавили балансировку данных - показатель roc_auc резко сократился с 0.85 до 0.49, но благодаря техники upsampled нам удалось вернуть его на уровень близкий к исходному (0.84).
Аналогично же и с accuracy. 

2) Случайный лес:

    - accuracy изменился с 0.86 (после балансировки - 0.6455)  до 0.8
    
    - f1 без изменений 0.59
    
    - roc_auc изменился с 0.85 (после балансировки - 0.6) до 0.85
    
3) Логистическая регрессия:
    
    - accuracy изменился с 0.79 (после балансировки 0.57) до 0.68
    
    - f1 без изменений 0.45
    
    - roc_auc изменился с 0.63 (после балансировки - 0.72) до 0.75


В связи с данными показателями можно сделать следующий вывод: хотя балансировка класса и позволяет нам увеличивать уровень метрики f1, но при этом после балансировки падают уровни метрики auc_roc и accuracy. Техника upsample же позволяет нам без изменений метрики f1 увеличить метрики auc_roc и accuracy до близкого к изначальному. 




In [51]:
# Создадим 3 сводных таблицы по разным метрикам
data_f1 = {'start':[0.56, 0.55, 0.067], 'after_balance':[0.57, 0.59, 0.45], 'after_upsampling':[0.57, 0.59, 0.45]}
data_accuracy = {'start':[0.86, 0.86, 0.79], 'after_balance':[0.6, 0.65, 0.57], 'after_upsampling':[0.77, 0.8, 0.68]}
data_roc_auc = {'start':[0.85, 0.85, 0.63], 'after_balance':[0.49, 0.6, 0.72], 'after_upsampling':[0.84, 0.85, 0.75]}
index = ['DecisionTreeClassifier', 'RandomForest', 'LogisticRegression']
data_f1 = pd.DataFrame(data_f1, index = index )
data_f1

,start,after_balance,after_upsampling
DecisionTreeClassifier,0.560,0.57,0.57
RandomForest,0.550,0.59,0.59
LogisticRegression,0.067,0.45,0.45


In [52]:
data_accuracy = pd.DataFrame(data_accuracy, index=index)
data_accuracy

,start,after_balance,after_upsampling
DecisionTreeClassifier,0.86,0.60,0.77
RandomForest,0.86,0.65,0.80
LogisticRegression,0.79,0.57,0.68


In [53]:
data_auc_roc = pd.DataFrame(data_roc_auc, index=index)
data_auc_roc

,start,after_balance,after_upsampling
DecisionTreeClassifier,0.85,0.49,0.84
RandomForest,0.85,0.60,0.85
LogisticRegression,0.63,0.72,0.75




Метрики у моделей после использования техники upsampled составили:

1) Дерево решений:

    - accuracy изначально показатель был 0.86, после балансировки стал 0.6, финально же составил 0.77
    
    - f1 без изменений (0.57)
    
    - roc_auc 0.84, тут без изменений
    
2) Случайный лес:

    - accuracy изменился с 0.86 (после балансировки - 0.6455)  до 0.8
    
    - f1 без изменений 0.59
    
    - roc_auc не изменился, значение составило 0.85 на всех промежутках
    
3) Логистическая регрессия:
    
    - accuracy изменился с 0.79 (после балансировки 0.57) до 0.68
    
    - f1 без изменений 0.45
    
    - roc_auc изменился с 0.63 (после балансировки - 0.75) до 0.75


Балансировка признаков позволяет нам увеличить значения метрики f1, но в то же время падает значение метрики accuracy. Тут нам помогла техника upsampling, в результате которой показатель accuracy удалось увеличить. 

Итог - техника upsampling помогает нам увеличить метрику accuracy, не изменяя метрики f1 и roc_auc. А значит - использование балансировки признаков без использования техники upsampling\downsampling лучше избежать, чтобы не потерять в качестве модели. 


In [49]:
data_roc_auc = {'start':[0.85, 0.85, 0.63], 'after_balance':[0.84, 0.85, 0.75], 'after_upsampling':[0.84, 0.85, 0.75]}
index = ['DecisionTreeClassifier', 'RandomForest', 'LogisticRegression']
data_auc_roc = pd.DataFrame(data_roc_auc, index=index)
data_auc_roc

,start,after_balance,after_upsampling
DecisionTreeClassifier,0.85,0.84,0.84
RandomForest,0.85,0.85,0.85
LogisticRegression,0.63,0.75,0.75


<a id='id_4'></a>
# 4. Тестирование модели

Проверим все 3 модели на тестовой выборке. 

In [93]:
# Дерево решений
model = DecisionTreeClassifier(max_depth=7, min_samples_leaf=4, random_state=123, class_weight='balanced')
model.fit(features_train, target_train)
tree_predict = model.predict(features_test)
f1_score(target_test, tree_predict)

0.58137347130762

In [94]:
accuracy_score(target_test, tree_predict)

0.7775

In [95]:
# Проверим разницу в отличии между нашей моделью и случайной выборкой
# Чем выше показатель - тем лучше
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_test, probabilities_one_valid)

0.8441780503238294

Выводы по дереву решеий:

    - f1 составляет 0.58
    
    - accuracy составялет 0.78
    
    - auc_roc составляет 0.84 (как известно - идеальное значние 1, а значит выборка к нему стремится)

In [96]:
# Случайный лес
model = RandomForestClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=2, n_estimators=10, random_state=123,
                              class_weight='balanced')
model.fit(features_train, target_train)
forest_predict = model.predict(features_test)
f1_score(target_test, forest_predict)

0.6027987082884821

In [97]:
accuracy_score(target_test, forest_predict)

0.8155

In [98]:
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_test, probabilities_one_valid)

0.8490634072805777

Выводы по Случайному лесу:

    - f1 составляет 0.60
    
    - accuracy составялет 0.81
    
    - auc_roc составляет 0.85 

In [99]:
# Логистическая регрессия
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
logistic_predict = model.predict(features_test)
f1_score(target_test, logistic_predict)

0.5083848190644307

In [100]:
accuracy_score(target_test, logistic_predict)

0.7215

In [101]:
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc_score(target_test, probabilities_one_valid)

0.7922591131293579

Выводы по Логистической регрессии:

    - f1 составляет 0.51
    
    - accuracy составялет 0.72
    
    - auc_roc составляет 0.79

## Вывод по пункту 4:

При проверке моделей на тестовой выборке показатели f1 составили:

    - Дерево решений 0.58
    
    - Случайный лес 0.60
    
    - Логистическая регрессия 0.51

При проверке моделей на тестовой выборке показатели accuracy составили:

    - Дерево решений 0.78
    
    - Случайный лес 0.81
    
    - Логистическая регрессия 0.72
    
При проверке моделей на тестовой выборке показатели auc_roc составили:

    - Дерево решений 0.84
    
    - Случайный лес 0.85
    
    - Логистическая регрессия 0.79    
    
    


In [63]:
# Проверим Дерево решений на тестовой выборке
tree_predict3 = tree_model2.predict(features_test)
f1_score(target_test, tree_predict3)

0.5860113421550094

In [64]:
accuracy_score(target_test, tree_predict3)

0.781

In [65]:
probabilities_test = tree_model2.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
roc_auc_score(target_test, probabilities_one_test)

0.843522792128837

In [66]:
# Случайный лес
forest_predict3 = forest_model2.predict(features_test)
f1_score(target_test, forest_predict3)

0.6056910569105691

In [67]:
accuracy_score(target_test, forest_predict3)

0.806

In [68]:
probabilities_test = forest_model2.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
roc_auc_score(target_test, probabilities_one_test)

0.8619398124022928

In [69]:
# Логистическая регрессия
logistic_predict3 = logistic_model2.predict(features_test)
f1_score(target_test, logistic_predict3)

0.5088339222614842

In [70]:
accuracy_score(target_test, logistic_predict3)

0.722

In [71]:
probabilities_test = logistic_model2.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
roc_auc_score(target_test, probabilities_one_test)

0.7922436040596541

### Вывод:

При проверке моделей на тестовой выборке после техники upsampling показатели f1 составили:

    - Дерево решений 0.58

    - Случайный лес 0.60

    - Логистическая регрессия 0.51

При проверке моделей на тестовой выборке после техники upsampling показатели accuracy составили:

    - Дерево решений 0.78

    - Случайный лес 0.81

    - Логистическая регрессия 0.72

При проверке моделей на тестовой выборке после техники upsampling показатели auc_roc составили:

    - Дерево решений 0.84

    - Случайный лес 0.85

    - Логистическая регрессия 0.75    

In [73]:
# Проверим корректность признаков
features_upsampled.info()
features_upsampled

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17151 entries, 7558 to 2575
Data columns (total 11 columns):
CreditScore          17151 non-null float64
Age                  17151 non-null float64
Tenure               17151 non-null float64
Balance              17151 non-null float64
NumOfProducts        17151 non-null float64
HasCrCard            17151 non-null float64
IsActiveMember       17151 non-null float64
EstimatedSalary      17151 non-null float64
Geography_Germany    17151 non-null uint8
Geography_Spain      17151 non-null uint8
Gender_Male          17151 non-null uint8
dtypes: float64(8), uint8(3)
memory usage: 1.2 MB


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7558,-0.291607,-0.279104,-1.449791,0.511571,0.799319,0.648425,-1.037018,0.195502,0,1,0
2160,0.794698,-1.027330,0.154019,-1.222776,0.799319,-1.542199,0.964303,-1.388338,0,0,1
2574,0.153260,1.123820,1.437068,0.537841,0.799319,-1.542199,0.964303,-1.709171,1,0,0
8940,0.070494,-0.092047,0.795544,-1.222776,0.799319,0.648425,-1.037018,1.487245,0,0,1
6295,-1.191689,1.030292,-1.129029,1.010300,2.510312,-1.542199,-1.037018,-0.525218,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2384,0.846427,0.095009,1.757830,-1.222776,0.799319,0.648425,-1.037018,0.939751,0,0,0
5922,-0.984774,0.469122,-0.487505,0.460806,2.510312,0.648425,-1.037018,-0.680912,0,0,0
3610,-0.726129,1.591461,-1.129029,0.868250,-0.911674,0.648425,-1.037018,0.121757,0,0,0
3127,1.301640,-0.559689,1.116306,0.609786,-0.911674,-1.542199,-1.037018,0.512958,1,0,1


In [83]:
target_upsampled.shape

(17151,)

Как можно увидеть - features_upsampled - признаки, которые были обработаны техниками масштабирования и upsampling.
Обучим этими признаками модели, которые создадим заново.

In [77]:
features_test.info()
features_test

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1314 to 7718
Data columns (total 11 columns):
CreditScore          2000 non-null float64
Age                  2000 non-null float64
Tenure               2000 non-null float64
Balance              2000 non-null float64
NumOfProducts        2000 non-null float64
HasCrCard            2000 non-null float64
IsActiveMember       2000 non-null float64
EstimatedSalary      2000 non-null float64
Geography_Germany    2000 non-null uint8
Geography_Spain      2000 non-null uint8
Gender_Male          2000 non-null uint8
dtypes: float64(8), uint8(3)
memory usage: 146.5 KB


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
1314,0.256718,1.404405,-0.808267,-1.222776,-0.911674,0.648425,-1.037018,0.860057,0,0,1
2588,0.939538,-0.653217,0.154019,-1.222776,-0.911674,0.648425,0.964303,-0.606411,0,0,1
6512,-1.688286,-1.401443,0.795544,0.904219,0.799319,0.648425,0.964303,-0.022714,0,0,1
6109,-0.715784,-1.214387,-0.487505,0.450121,-0.911674,0.648425,0.964303,1.464615,1,0,0
2259,0.370521,0.188537,-0.166743,0.839367,0.799319,0.648425,-1.037018,1.517910,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4112,0.515362,-0.840273,1.437068,-1.222776,-0.911674,0.648425,0.964303,1.299385,0,0,0
2055,-1.874509,-0.092047,1.437068,0.585418,-0.911674,-1.542199,-1.037018,-1.521632,1,0,1
2307,-0.415757,1.684990,0.474781,0.479388,0.799319,-1.542199,0.964303,1.500825,1,0,1
6502,-0.405411,0.095009,-0.808267,0.780880,-0.911674,0.648425,-1.037018,-0.115580,0,0,0


In [74]:
# Дерево решений (обучим его заново)
model = DecisionTreeClassifier(max_depth=7, min_samples_leaf=4, random_state=123, class_weight='balanced')
model.fit(features_upsampled, target_upsampled)
predict = model.predict(features_test)
f1_score(target_test, predict)

0.5860113421550094

In [84]:
accuracy_score(target_test, predict)

0.781

In [85]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
roc_auc_score(target_test, probabilities_one_test)

0.843522792128837

In [86]:
# Дерево решений без upsampling 
model = DecisionTreeClassifier(max_depth=7, min_samples_leaf=4, random_state=123, class_weight='balanced')
model.fit(features_train, target_train)
predict = model.predict(features_test)
f1_score(target_test, predict)

0.58137347130762

In [87]:
accuracy_score(target_test, predict)

0.7775

In [88]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
roc_auc_score(target_test, probabilities_one_test)

0.8441780503238294

### Вывод:
На примере Дерева решений мы можем убедиться, что изначально данные были корректными, просто разница между метриками на выборках с апсемплингом и без него очень мизерная, что при округлении до двух сотых - её практически не видно. 

На примере метрик:
    
    - f1 0.5860113421550094 и 0.58137347130762
    
    - accuracy 0.781 и 0.7775
    
    - roc_auc 0.843522792128837 и 0.8441780503238294

<a id='id_1'></a>
# 5. Общий вывод:

Наиболее подходящей моделью является Случайный лес. Его показатели составили:
    
    - f1 равен 0.6
    
    - accuracy равно 0.81
    
    - auc_roc равен 0.85
    
Случайные лес лидирует по всем показателям, а значит точность ответов, а так же отношение точности и полноты у данной модели самые высокие. Высокий уровень auc_roc говорит так же о маленькой зависимости от случайной модели. 

Модель Дерево решений подходит нам чуть меньше (f1 0.58, accuracy 0.78, auc_roc 0.84). Разница со Случайным лесом сравнительно маленькая, а значит, как известно из теории - модель Дерево решений наиболее быстрое, и если возникнут проблемы со скоростью обработки данных моделью Случайный лес - модель Дерево решений так же подойдет, но со сравнительно небольшой потерей качества. 
Но по условиям задачи f1 должен быть выше 0.58, а значит модель Дерево решений в контексте данной задачи нам не подходит. 

Модель Логистическая регрессия проигрываем по всем параметрам и отвергается нами.

___Вывод - подходит только модель Случайный лес___. 